Import de packages "externes"

In [2]:
import time
import pandas as pd
import numpy as np
import random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

Import de fonctions de auxiliary.py

In [3]:
from auxiliary_functions import remove_zero_start
from auxiliary_functions import intersect_list
from auxiliary_functions import complement_list
from auxiliary_functions import split_list
from auxiliary_functions import empty_df

In [4]:
titles = ["Name", "Expansion", "Min price", "Exemplaires en vente", "Rareté", "mkm_url", "expansion_release_date"]

Ne PAS LANCER (reset le tableau de données)

In [3]:
data = pd.DataFrame({})

titles = ["Name", "Expansion", "Min price", "Exemplaires en vente", "Rareté", "mkm_url", "expansion_release_date"]
for title in titles:
    data[title] = []

#data.to_csv('original_data.csv', index=False)


In [ ]:
chrome_options = webdriver.ChromeOptions()
#Maximise la taille de la fenetre ouverte
chrome_options.add_argument("--start-maximized")
#Evite des bugs
chrome_options.add_argument("--disable-gpu")
#Lance chrome sans interface ouverte (il tourne en fond)
#chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
#Change la gestion de la mémoire, peut régler certains bugs
#chrome_options.add_argument("--disable-dev-shm-usage")

cService = webdriver.ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service = cService, options=chrome_options)


expansion_list = ["Lost-Origin", "Stellar-Crown", "Surging-Sparks", "Shrouded-Fable", "Twilight-Masquerade", "Temporal-Forces", "Paldean-Fates", "Paradox-Rift", "151", "Obsidian-Flames", "Paldea-Evolved", "Scarlet-Violet"]




def add_expansion(expansion_name):
    dico = {}
    data_lign = 0
    base_url = "https://www.cardmarket.com/en/Pokemon/Products/Singles/"
    url_extension = "https://www.cardmarket.com/en/Pokemon/Expansions/" + expansion_name + "#SetInfoSection"
    driver.get(url_extension)
    time.sleep(random.uniform(2, 5))
    release_date = driver.find_element(By.XPATH, "/html/body/main/div[2]/div[2]/div/div[1]/div[2]").text

    #On récupère le nombre de page de cartes dans l'extension choisie
    driver.get(base_url + expansion_name)
    time.sleep(random.uniform(2, 5))
    nmbr_page = int(driver.find_element(By.CLASS_NAME, "mx-1").text[-3:].replace(" ", "").replace("+", ""))

    for i in range(1, nmbr_page+1):
        time.sleep(random.uniform(2, 5))
        driver.get(base_url+ expansion_name + "?site=" + str(i))
        suffixe = "productRow"
        elements = driver.find_elements(By.XPATH, f"//*[starts-with(@id, '{suffixe}')]")
        for element in elements:
            card_elements = []

            #On récupère le nom de la carte
            card_name = element.find_elements(By.TAG_NAME, "a")[1].text
            card_elements.append(card_name)

            #On rajoute le nom de l'extension
            card_elements.append(expansion_name)

            #On récupère le prix minimum de vente
            min_price = element.find_element(By.CLASS_NAME, "col-price.pe-sm-2").text
            card_elements.append(min_price) 

            #On récupère le nombre d'exemplaires disponibles
            dispo = element.find_element(By.CLASS_NAME, "d-none.d-md-inline").text
            card_elements.append(dispo) 

            #On récupère la rareté
            rarete = element.find_element(By.TAG_NAME, "svg").get_attribute("aria-label")
            card_elements.append(rarete)


            #On récupère l'url de la carte sur mkm
            mkm_url = element.find_elements(By.TAG_NAME, "a")[1].get_attribute("href")
            card_elements.append(mkm_url)    

            #On rajoute la date de sortie de l'extension
            card_elements.append(release_date)  

            #On rajoute à l'entrée "nom de la carte" ses caractéristiques dans le dico
            dico[data_lign] = card_elements
            data_lign+=1
    data = pd.DataFrame.from_dict(dico, orient = "index")
    data.columns = titles
    existent_data = pd.read_csv('original_data.csv')
    new_data = pd.concat([existent_data, data], ignore_index=True)
    new_data.to_csv('original_data.csv', index=False)

#add_expansion(expansion_list[11])

driver.quit()

On fait une copie des données originales

In [9]:
data = pd.read_csv('original_data.csv')
data.to_csv('working_data.csv', index=False)

On travail sur les données.
On travaillera toujours sur working_data.csv qui est une copie de original_data. En effet, on ne veut pas perdre ou modifier notre tableau de donnée initial issue du scrapping.

In [10]:
data = pd.read_csv('working_data.csv')

In [11]:
data.head()

,Name,Expansion,Min price,Exemplaires en vente,Rareté,mkm_url,expansion_release_date,Tournament_last_month,Price trend,Price 7 days,Price 30 days
0,Live Code Card (Booster) (LOR),Lost-Origin,"0,02 €",14753,Online Code Card,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",NaN,NaN,NaN,NaN
1,Thorton (LOR 167),Lost-Origin,"0,02 €",8496,Uncommon,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",NaN,NaN,NaN,NaN
2,Rotom V (LOR 058),Lost-Origin,"0,70 €",239,Ultra Rare,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",193.0,"3,69 €","3,56 €","3,23 €"
3,Colress's Experiment (LOR 155),Lost-Origin,"0,02 €",6339,Uncommon,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",NaN,NaN,NaN,NaN
4,Lost Vacuum (LOR 162),Lost-Origin,"0,02 €",12538,Uncommon,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",NaN,NaN,NaN,NaN


On enlève les cartes qui correspondent à des codes en ligne, les cartes oversized (non jouables), et les cartes pour lesquelles il manque une information.
En l'occurence, il n'y a qu'une seule carte dans cette dernière catégorie, qui n'existe pas en physique (bug de mkm).
Un rapide descriptif des raretés:
    -Common/Uncommon/Rare/Ultra Rare: raretés "classiques" par ordre croissant, disponibles dans toutes les extensions (récentes du moins)
    -Illustration Rare: illustration alternatives
    -Special illustration Rare: illustration alternatives, mais de cartes initialements plus rare. (attention, les cartes ayant une illustration alternative n'ont pas forcément d'illustration "de base")
    -Secret rare: illustrations particulière, très peu fréquentes
    -Promo: cartes promotionelles
    -Shiny rare/ Shiny Ultra Rare: versions alternatives des rares/Ultra rares avec une couleur différente

Toutes les raretés ne sont pas présentes dans toutes les extensions.

In [6]:
data = data[(data["Rareté"] != "Online Code Card") & (data["Rareté"] != "Oversized")]
data = data.dropna(subset=["Min price"])

On enlève les cartes qui coutent moins qu'un certain prix (50 centimes ici). 
On modifie la colonne "Min price" en retirant le symbole euro pour juste laisser le prix au format AAA.BB en type int (et plus str)

In [7]:
data["Min price"] = (data['Min price'].str[:-5] + data['Min price'].str[-4:-2]).astype(int)/100

In [8]:
data = data[(data["Min price"] >= 0.5)]

On sépare le nom en deux colonnes: le nom effectif et le "tag" qui correspond à l'extension et au numéro.

In [186]:
data[["Name", "Code"]] = data["Name"].str.split(" \(", expand = True)
data ["Code"] = data["Code"].str.rstrip(")")

Toutes les étapes ci dessus en une fois

In [7]:
data = pd.read_csv('working_data.csv')

data = data[(data["Rareté"] != "Online Code Card") & (data["Rareté"] != "Oversized")]
data = data.dropna(subset=["Min price"])

data["Min price"] = (data['Min price'].str[:-5] + data['Min price'].str[-4:-2]).astype(int)/100

data = data[(data["Min price"] >= 0.5)]

data[["Name", "Code"]] = data["Name"].str.split(" \(", expand = True)
data["Code"] = data["Code"].str.rstrip(")")
data[["Expansion_code", "Number_code"]] = data["Code"].str.split(" ", expand = True)
data["Number_code"] = data["Number_code"].apply(remove_zero_start)

On scrap "https://limitlesstcg.com/" pour récupérer le nombre d'utilisation en tournoi dans le mois passé de certaines cartes d'intérêt.
Cette variable, bien qu'imparfaite, donne une variable exploitable pour expliquer le prix des cartes.
On s'intéressera a priori à terme uniquement aux cartes "ultra rare". 
En effet, la rareté supérieure des cartes (secret rare ou autre) n'est a priori pas une variable d'intérêt quand on veut juste jouer la carte.

In [ ]:
indexes = data.index[data["Rareté"] == "Ultra Rare"].to_list()
original_data = pd.read_csv("original_data.csv")


chrome_options = webdriver.ChromeOptions()
#Maximise la taille de la fenetre ouverte
chrome_options.add_argument("--start-maximized")
#Evite des bugs
chrome_options.add_argument("--disable-gpu")
#Lance chrome sans interface ouverte (il tourne en fond)
#chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
#Change la gestion de la mémoire, peut régler certains bugs
#chrome_options.add_argument("--disable-dev-shm-usage")

cService = webdriver.ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service = cService, options=chrome_options)

def add_tournament_use(index_list):
    base_url = "https://limitlesstcg.com/cards/"
    for index in index_list:
        lign = data.loc[index]
        expansion = lign["Expansion_code"]
        code = lign["Number_code"]
        url = base_url + expansion + "/" + code + "/decklists?time=1months&type=all&format=all&region=all&division=all"
        driver.get(url)
        time.sleep(random.uniform(3, 6))

        table = driver.find_element(By.CLASS_NAME, "data-table.striped")
        tournament_use = len(table.find_elements(By.TAG_NAME, "tr"))-1
        original_data.loc[index, "Tournament_last_month"] = tournament_use
    original_data.to_csv("original_data.csv", index=False)



add_tournament_use(indexes[])

driver.quit()

In [15]:
data = pd.read_csv('original_data.csv')
data.to_csv('working_data.csv', index=False)

data = pd.read_csv('working_data.csv')

data = data[(data["Rareté"] != "Online Code Card") & (data["Rareté"] != "Oversized")]
data = data.dropna(subset=["Min price"])


data["Min price"] = (data['Min price'].str[:-5] + data['Min price'].str[-4:-2]).astype(int)/100

data = data[(data["Min price"] >= 0.5)]

#On corrige la typo de "Secret Rare" qui est écrite avec un " " à la fin
data.loc[data["Rareté"] == "Secret Rare ", "Rareté"] = "Secret Rare"

data[["Name", "Code"]] = data["Name"].str.split(" \(", expand = True)
data["Code"] = data["Code"].str.rstrip(")")
data[["Expansion_code", "Number_code"]] = data["Code"].str.split(" ", expand = True)
data["Number_code"] = data["Number_code"].apply(remove_zero_start)

In [ ]:
rarity = "Secret Rare"
index = data.index[data["Rareté"] == rarity].to_list()

indexes_list = split_list(index, step = 15)

On scrappe plus en détail certaines cartes d'intérêt sur MKM.
On récupère en particulier le "price trend", le "prix 7 jours" et le "prix 30 jours" qui sont les valeurs moyennes auxquelles les cartes sont échangées dans les périodes de temps données.
Celles ci sont plus intéressante que le prix mini (potentiellement des cartes en mauvais état, ou qui présentes des facteurs désavantageux, par exemple des frais de port)

In [14]:
for indexes in indexes_list:

    original_data = pd.read_csv("original_data.csv")

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-gpu")
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")

    cService = webdriver.ChromeService(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service = cService, options=chrome_options)

    def add_price_trends(indexes):
        """
        Prend en argument une liste d'index, et rajoute les tendances de prix de toutes les cartes correspondantes.
        On utilise pour cela l'URL récupéré initialement dans le tableau.
        """
        for index in indexes:
            lign = data.loc[index]
            url = lign["mkm_url"]
            driver.get(url)
            time.sleep(random.uniform(3, 6))

            price_trend = driver.find_element(By.XPATH, '//*[@id="tabContent-info"]/div/div[1]/div/div[2]/dl/dd[8]/span').text
            price_30 = driver.find_element(By.XPATH, '//*[@id="tabContent-info"]/div/div[1]/div/div[2]/dl/dd[9]/span').text
            price_7 = driver.find_element(By.XPATH, '//*[@id="tabContent-info"]/div/div[1]/div/div[2]/dl/dd[10]/span').text
            original_data.loc[index, "Price trend"] = price_trend
            original_data.loc[index, "Price 7 days"] = price_7
            original_data.loc[index, "Price 30 days"] = price_30

        original_data.to_csv("original_data.csv", index = False)
        
        driver.quit()

    add_price_trends(indexes)

In [18]:
data[data["Rareté"] == "Secret Rare"]

,Name,Expansion,Min price,Exemplaires en vente,Rareté,mkm_url,expansion_release_date,Tournament_last_month,Price trend,Price 7 days,Price 30 days,Code,Expansion_code,Number_code
49,Pikachu VMAX,Lost-Origin,5.00,577,Secret Rare,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",NaN,"7,76 €","7,95 €","7,23 €",LOR TG29,LOR,TG29
54,Mew VMAX,Lost-Origin,4.99,488,Secret Rare,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",NaN,"6,74 €","6,84 €","7,00 €",LOR TG30,LOR,TG30
140,Giratina VSTAR,Lost-Origin,7.50,317,Secret Rare,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",NaN,"14,28 €","13,91 €","13,84 €",LOR 212,LOR,212
160,Lost Vacuum,Lost-Origin,1.49,325,Secret Rare,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",NaN,"5,76 €","7,30 €","5,66 €",LOR 217,LOR,217
194,Collapsed Stadium,Lost-Origin,4.20,237,Secret Rare,https://www.cardmarket.com/en/Pokemon/Products...,"9TH SEPTEMBER, 2022",NaN,"6,66 €","5,50 €","5,40 €",LOR 215,LOR,215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2758,Basic Lightning Energy,Scarlet-Violet,2.99,318,Secret Rare,https://www.cardmarket.com/en/Pokemon/Products...,"31ST MARCH, 2023",NaN,"4,74 €","5,97 €","4,81 €",SVI 257,SVI,257
2767,Miraidon ex,Scarlet-Violet,4.00,445,Secret Rare,https://www.cardmarket.com/en/Pokemon/Products...,"31ST MARCH, 2023",NaN,"7,86 €","7,62 €","7,80 €",SVI 253,SVI,253
2773,Rare Candy,Scarlet-Violet,5.00,287,Secret Rare,https://www.cardmarket.com/en/Pokemon/Products...,"31ST MARCH, 2023",NaN,"9,81 €","8,04 €","9,43 €",SVI 256,SVI,256
2804,Koraidon ex,Scarlet-Violet,3.90,552,Secret Rare,https://www.cardmarket.com/en/Pokemon/Products...,"31ST MARCH, 2023",NaN,"6,33 €","6,04 €","6,40 €",SVI 254,SVI,254
